In [1]:
import netCDF4

from read_mira_radar import read_mira


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [3]:
data_path = "/home/cclopes/git/amazon-storms-aerosols/data/radar/mira_campina/"

# mira = read_mira(data_path + "2020/08/05/05/20200805_1200.mmclx")[0]
mira = netCDF4.Dataset(data_path + "2020/08/05/05/20200805_1200.mmclx")

/home/cclopes/miniconda3/envs/amazon-storms-aerosols/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# for var in mira.variables.keys():
#     print(mira.variables[var].long_name)

print(mira)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    convention: CF-1.0
    location: ATTO Campina
    Altitude: 37m
    Latitude: 02.1814S
    Longitude: 59.0218W
    system: C5XC
    title: MIRA Cloud Radar Data
    institution: USP
    source: 200805_120003.pds.off
    reference: Ka Band Cloud Radar MIRA, METEK GmbH www.metek.de
    ppar: PRF:        5000
PDR:         208
SFT:         256
AVC:         200
IHP:           2
CHG:         575
POL:         208
ATT:        1500
TX:           0
ADCGAIN0:      0.00000
ADCGAIN1:      0.00000
WND:           0
POS: -2147483424
ADD:          48
LEN:           0
CAL:          96
NOS: -2147483648
OF0:          40
OF1:           0
SWT:           0
SUM:           0
OSC:          26
TST:           0
COR:           1
OFS:           0
HSBN:           1
HSBA:           1
CALIBRPOWER_M:      1.00000
CALIBRSNR_M:      281.838
CALIBRPOWER_S:      1.00000
CALIBRSNR_S:      234.423
RAW_GATE1:           2
RAW_G

/home/cclopes/miniconda3/envs/amazon-storms-aerosols/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
